In [1]:
import pandas as pd

# Paths to your Excel files
file_paths = {
    "tts_details": "D:/Akash_Tripathi/Data/TTsDetails.xlsx",
    "night_violation": "D:/Akash_Tripathi/Data/NightViolation.xlsx",
    "speed_violation": "D:/Akash_Tripathi/Data/SpeedViolation.xlsx",
    "route_violation": "D:/Akash_Tripathi/Data/RouteViolation.xlsx",
    "stoppage_violation": "D:/Akash_Tripathi/Data/StoppageViolation.xlsx",
    "vehicle_invoices": "D:/Akash_Tripathi/Data/INVOICE_TRIP_MASTER.xlsx"
}

# Load data into DataFrames
tts_details = pd.read_excel(file_paths["tts_details"])
night_violation = pd.read_excel(file_paths["night_violation"])
speed_violation = pd.read_excel(file_paths["speed_violation"])
route_violation = pd.read_excel(file_paths["route_violation"])
stoppage_violation = pd.read_excel(file_paths["stoppage_violation"])
vehicle_invoices = pd.read_excel(file_paths["vehicle_invoices"])

In [4]:
# Aggregating Route Violations
route_violations = route_violation.merge(vehicle_invoices, left_on='IdInvTripMstr', right_on='IdInvTripMstr')
route_violations = route_violations.merge(tts_details, left_on='VehicleNo', right_on='AssignVehNo')
route_violations_count = route_violations.groupby(['VehicleNo', 'TransportersName']).size().reset_index(name='Route_Violation_Count')

# Aggregating Speed Violations
speed_violations = speed_violation.merge(vehicle_invoices, on='IdInvTripMstr', how='left')
speed_violations = speed_violations.merge(tts_details, left_on='VehicleNo', right_on='AssignVehNo', how='left')
speed_violations_count = speed_violations.groupby('TransportersName')['IdInvTripMstr'].count().reset_index(name='Speed_Violation_Count')

# Aggregating Stoppage Violations (New Logic)
stoppage_violation_count = stoppage_violation.groupby('TransportersName')['IdInvTripMstr'].count().reset_index(name='Stoppage_Violation_Count')

# Aggregating Night Violations
night_violations = night_violation.merge(vehicle_invoices, on='IdInvTripMstr', how='left')
night_violations = night_violations.merge(tts_details, left_on='VehicleNo', right_on='AssignVehNo', how='left')
night_violations_count = night_violations.groupby('TransportersName')['IdInvTripMstr'].count().reset_index(name='Night_Violation_Count')

invoice_count = vehicle_invoices.merge(tts_details, left_on='VehicleNo', right_on='AssignVehNo')
invoice_count = invoice_count.groupby(['VehicleNo', 'TransportersName']).size().reset_index(name='Invoice_Count')

# Aggregating Truck Counts
truck_count = tts_details.groupby('TransportersName')['AssignVehNo'].nunique().reset_index(name='Truck_Count')


In [5]:
# Combine all the aggregated data
aggregated_violations = truck_count.merge(invoice_count[['TransportersName', 'Invoice_Count']], on='TransportersName', how='left')
aggregated_violations = aggregated_violations.merge(route_violations_count, on='TransportersName', how='left')
aggregated_violations = aggregated_violations.merge(speed_violations_count, on='TransportersName', how='left')
aggregated_violations = aggregated_violations.merge(stoppage_violation_count, on='TransportersName', how='left')
aggregated_violations = aggregated_violations.merge(night_violations_count, on='TransportersName', how='left')

In [6]:
aggregated_violations.fillna(0, inplace=True)

In [7]:
# Summing up the violation counts
aggregated_violations = aggregated_violations.groupby('TransportersName').agg({
    'Truck_Count': 'sum',
    'Invoice_Count': 'sum',
    'Route_Violation_Count': 'sum',
    'Speed_Violation_Count': 'sum',
    'Stoppage_Violation_Count': 'sum',
    'Night_Violation_Count': 'sum'
}).reset_index()

In [8]:
# Calculate total and average violations per truck
aggregated_violations['Total_Violations'] = (aggregated_violations['Route_Violation_Count'] +
                                             aggregated_violations['Speed_Violation_Count'] +
                                             aggregated_violations['Stoppage_Violation_Count'] +
                                             aggregated_violations['Night_Violation_Count'])

aggregated_violations['AverageViolationsPerTruck'] = aggregated_violations['Total_Violations'] / aggregated_violations['Truck_Count']

In [9]:
# Filter out transporters with zero average violations per truck
aggregated_violations = aggregated_violations[aggregated_violations['AverageViolationsPerTruck'] > 0]

In [15]:
# Identify Top 10 Transporters
top_transporters = aggregated_violations.sort_values(by='AverageViolationsPerTruck').head(10)

In [16]:
# Print results
print(top_transporters[['TransportersName', 'AverageViolationsPerTruck']])

                   TransportersName  AverageViolationsPerTruck
323                    UPHAR INDANE                   2.000000
297      SRI PERIYANDAVAR TRANSPORT                   2.000000
75        DOMESTIC GAS & APPLIANCES                   5.666667
36             AWADHESH KUMAR SINGH                   5.814815
32                   ARYA TRANSPORT                   6.000000
4                ADITYA ENTERPRISES                   7.928571
216  RAMAWATAR INDANE GRAMIN VITRAK                  10.000000
17                       ANIL KUMAR                  10.750000
224           RAVI TRANSPORT AGENCY                  12.866667
291            SOUMITRA ENTERPRISES                  13.666667


In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming 'aggregated_violations' DataFrame is already created and cleaned

# Create labels for top 10 transporters
aggregated_violations['Top_10'] = aggregated_violations['AverageViolationsPerTruck'].rank(method='first', ascending=True) <= 10

# Prepare features and labels
features = aggregated_violations[['Truck_Count', 'Invoice_Count', 'Route_Violation_Count', 'Speed_Violation_Count', 'Stoppage_Violation_Count', 'Night_Violation_Count']]
labels = aggregated_violations['Top_10'].astype(int)  # Convert boolean to int (1 for top 10, 0 for others)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Initialize and train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)
print('Confusion Matrix:')
print(conf_matrix)

# To make predictions on new data
new_data = pd.DataFrame({
    'Truck_Count': [10],
    'Invoice_Count': [100],
    'Route_Violation_Count': [2],
    'Speed_Violation_Count': [2],
    'Stoppage_Violation_Count': [3],
    'Night_Violation_Count': [5]
})
prediction = model.predict(new_data)
print('Prediction:', prediction)

Accuracy: 0.99
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        66
           1       0.00      0.00      0.00         1

    accuracy                           0.99        67
   macro avg       0.49      0.50      0.50        67
weighted avg       0.97      0.99      0.98        67

Confusion Matrix:
[[66  0]
 [ 1  0]]
Prediction: [1]


C:\Users\MLINF\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\MLINF\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\MLINF\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo